# Datawarehouse creation & ETL dev Environment

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Connect to S3 and explore the file source data
from time import time
import configparser
import matplotlib.pyplot as plt
import pandas as pd
import sql_queries
import create_dwh
import delete_dwh
import create_tables
import etl
from smart_open import smart_open


config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
KEY=config.get('AWS','key')
SECRET= config.get('AWS','secret')
LOG_DATA= config.get('S3', 'log_data') 
LOG_JSONPATH= config.get('S3', 'log_jsonpath')
SONG_DATA= config.get('S3', 'song_data')
print(LOG_DATA)

's3://udacity-dend/log_data'


In [13]:
# Create Redshift Cluster
%run -i create_dwh

1.1 Creating a new IAM Role
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name dwhRole already exists.
Attaching S3 Policy
Policy Attached
Attaching Redshift Policy
Policy Attached
1.3 Get the IAM role ARN
Cluster Created


In [16]:
# Create Tables
%run -i create_tables
# first time error: string indices must be integer, then it works after..
# fails until the cluster exists, actually need to put a 'wait'

dwhcluster.ctm8gofjtwwi.us-west-2.redshift.amazonaws.com
Connected to: dwh
Now executing: CREATE TABLE IF NOT EXISTS staging_events (
    artist text,
    auth text,
    firstName text,
    gender text,
    itemInSession text,
    lastName text,
    length float,
    level text,
    location text,
    method text,
    page text,
    registration text,
    sessionId integer,
    song text,
    status text,
    ts timestamp,
    userAgent text,
    userID int
)
;
Now executing: CREATE TABLE IF NOT EXISTS staging_songs (
    num_songs text,
    artist_id text,
    latitude numeric,
    longitude numeric,
    location text,
    name text,
    song_id text,
    title text,
    duration float,
    year integer
)
;
Now executing: 
    CREATE TABLE IF NOT EXISTS users (
    user_id int sortkey distkey,
    first_name varchar,
    last_name varchar,
    gender varchar, 
    level varchar
    )
;
Now executing: 
    CREATE TABLE IF NOT EXISTS songs (
    song_id varchar,
    title varchar,
    a

In [17]:
# Copy from S3 to staging & execute one insert statement
%run -i etl

Connected to: dwh
Copy Query Completed:copy staging_events from 's3://udacity-dend/log_data'
                            credentials 'aws_iam_role=arn:aws:iam::230036167675:role/dwhRole'
                             region 'us-west-2'
                             timeformat as 'epochmillisecs'
                             truncatecolumns blanksasnull emptyasnull
                             json 's3://udacity-dend/log_json_path.json'  
;
Copy Query Completed:copy staging_songs from 's3://udacity-dend/song_data'
                            credentials 'aws_iam_role=arn:aws:iam::230036167675:role/dwhRole'
                            format as json 'auto' 
                            region 'us-west-2'
;
Insert Query Completed:
INSERT INTO songs (song_id, title, artist_id, year, duration) 
SELECT 
    song_id, 
    title, 
    artist_id, 
    year, 
    duration
FROM staging_songs
;
Insert Query Completed:
INSERT INTO artists (artist_id, name, location, latitude, longitude) 
SELECT
    ar

In [10]:
# Delete cluster
%run -i delete_dwh

An error occurred (ClusterNotFound) when calling the DeleteCluster operation: Cluster dwhcluster not found.
